Q: [1008. Construct Binary Search Tree from Preorder Traversal](https://leetcode.com/problems/construct-binary-search-tree-from-preorder-traversal/)

**Approach 1: Iterative.**
* `preorder[0]` is the root (by definition of preorder traversal).
* The tree is built depth-first-- first the left subtree and then the right one.
* Consider building a tree rooted at a node, say, `n`. Having built its left subtree, to be able to access `n` again, so that its right subtree can be built, a stack must be used.
* While building a subtree, elements are pushed onto the stack. While traversing back to get the subtree's root (to start building the right subtree), elements are popped from the stack.

In [ ]:
# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right
class Solution:
    def bstFromPreorder(self, preorder: List[int]) -> Optional[TreeNode]:
        root = TreeNode(preorder[0])
        stack = [root]
        
        for val in preorder[1:]:
            if val < stack[-1].val:
                stack[-1].left = TreeNode(val)
                stack.append(stack[-1].left)                
                
            else:
                while stack and val > stack[-1].val:
                    temp = stack.pop()
                sub_root = temp
                sub_root.right = TreeNode(val)
                stack.append(sub_root.right)                
                
        return root

**Approach 2: Recursive**

* With `preorder[0]` as the root, find `mid` such that `val <= preorder[0] for val in preorder[1:mid]` and `val > preorder[0] for val in preorder[mid:]` (see [bisect.bisect](https://docs.python.org/3/library/bisect.html#bisect.bisect)).
* Values in `preorder[1:mid]` form the left subtree, and values in `preorder[mid:]` form the right subtree.

* With minor tweaks to this general idea, we can have the following approaches:



**2.1**

* `right_subroot` corresponds to `mid` described above.
* For why *right*, see doc for the difference between `bisect_left` and `bisect`.

In [ ]:
# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right
class Solution:
    def bstFromPreorder(self, preorder: List[int]) -> Optional[TreeNode]:
        if not preorder:
            return None
        root = TreeNode(preorder[0])
        right_subroot = bisect.bisect(preorder, preorder[0])
        root.left = self.bstFromPreorder(preorder[1:right_subroot])
        root.right = self.bstFromPreorder(preorder[right_subroot:])
        
        return root

**2.2**

* Instead of slicing the list in each call, we use two pointers.

In [ ]:
# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right
class Solution:
    def bstFromPreorder(self, preorder: List[int]) -> Optional[TreeNode]:
        def build(i, j):
            if i == j:
                return None
            root = TreeNode(preorder[i])
            mid = bisect.bisect(preorder, preorder[i], i, j)
            root.left = build(i+1, mid)
            root.right = build(mid, j)            
            return root
        
        return build(0, len(preorder))

**2.3**

* Instead of calling `bisect` in each call, we sort the elements inorder, and construct an `inorder_map`-- {(node.val, inorder position)}.
* At each node in `preorder`, build the subtree rooted at it.

In [ ]:
# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right
class Solution:
    def bstFromPreorder(self, preorder: List[int]) -> Optional[TreeNode]:
        inorder = sorted(preorder)
        self.preInd = 0
        inorder_map = {}
        for i, x in enumerate(inorder):
            inorder_map[x] = i
        
        def solve(start, end):
            if start > end:
                return None
            rootVal = preorder[self.preInd]
            self.preInd += 1            
            root = TreeNode(rootVal)            
            root.left = solve(start, inorder_map[rootVal]-1)
            root.right = solve(inorder_map[rootVal]+1, end)
            return root
        
        return solve(0, len(preorder)-1)